In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [2]:
THRESH_SPO2_LOW = 92
THRESH_SPO2_CRITICAL = 88
THRESH_SHOCK_INDEX_HIGH = 0.9
THRESH_SHOCK_INDEX_CRITICAL = 1.0
THRESH_MAP_LOW = 65  # Mean Arterial Pressure threshold for shock
THRESH_LACTATE_HIGH = 2.0  # mmol/L for tissue hypoperfusion
THRESH_LACTATE_CRITICAL = 4.0
THRESH_UO_LOW = 0.5  # mL/kg/hr (oliguria)

# Trend Analysis
TREND_WINDOW = 6  # Number of readings for short-term trend analysis


# AGE-SPECIFIC VITAL SIGN THRESHOLDS (Low, Normal, High)

AGE_THRESHOLDS = {
    'neonate': {
        'rr_low': 30, 'rr_normal': 40, 'rr_high': 60,
        'hr_low': 100, 'hr_normal': 140, 'hr_high': 160,
        'sbp_low': 60, 'sbp_normal': 70, 'sbp_high': 90,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'infant': {
        'rr_low': 24, 'rr_normal': 30, 'rr_high': 40,
        'hr_low': 80, 'hr_normal': 120, 'hr_high': 140,
        'sbp_low': 70, 'sbp_normal': 85, 'sbp_high': 100,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'child': {
        'rr_low': 16, 'rr_normal': 20, 'rr_high': 30,
        'hr_low': 70, 'hr_normal': 90, 'hr_high': 110,
        'sbp_low': 80, 'sbp_normal': 95, 'sbp_high': 110,
        'temp_low': 36.0, 'temp_normal': 37.0, 'temp_high': 38.0
    },
    'adolescent': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 75, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 105, 'sbp_high': 120,
        'temp_low': 35.8, 'temp_normal': 36.8, 'temp_high': 37.8
    },
    'adult': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 80, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 115, 'sbp_high': 130,
        'temp_low': 35.5, 'temp_normal': 36.8, 'temp_high': 38.0
    },
    'geriatric': {  # Added for elderly patients who may have different baselines
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 24, # Often higher RR baseline
        'hr_low': 55, 'hr_normal': 70, 'hr_high': 90, # Often lower HR
        'sbp_low': 90, 'sbp_normal': 125, 'sbp_high': 140, # Often higher SBP
        'temp_low': 35.5, 'temp_normal': 36.5, 'temp_high': 37.5 # Often lower temp
    }
}


SCORE_THRESHOLDS = {
    "hypotensive_shock": {  # Added for the new pipeline
        "yellow": 3,   # Monitor/Review
        "orange": 6,   # Urgent
        "red": 9       # Critical
    },
}


In [3]:
def assign_age_category(df):
    """
    Assigns an age category based on the 'age' column in the DataFrame.
    Now includes more granular pediatric categories and a geriatric category.
    """
    df = df.copy()
    
    def _categorize(age):
        if age <= 0.083: return 'neonate'     # < 1 month
        elif age <= 1:   return 'infant'      # 1 month - 1 year
        elif age < 5:    return 'child'       # 1 - 5 years
        elif age < 13:   return 'adolescent'  # 5 - 12 years
        elif age < 65:   return 'adult'       # 13 - 64 years
        else:            return 'geriatric'   # 65+ years
    
    if 'age' in df.columns:
        df['age_category'] = df['age'].apply(_categorize)
    else:
        # Default to adult if age is not provided
        df['age_category'] = 'adult'
    
    return df

In [4]:
def apply_vital_range_flags(df):
    """
    Applies age-specific thresholds to flag abnormal vital signs.
    Now includes flags for all parameters needed across pipelines.
    """
    df = df.copy()
    df = assign_age_category(df)  # Inject age group

    # SpO₂ flags (Absolute threshold)
    df['flag_spo2_low'] = df['spo2'] < THRESH_SPO2_LOW
    df['flag_spo2_critical'] = df['spo2'] < THRESH_SPO2_CRITICAL

    # Temperature flags
    df['flag_temp_high'] = df.apply(lambda row: row['temperature'] >= AGE_THRESHOLDS[row['age_category']]['temp_high'], axis=1)
    df['flag_temp_low'] = df.apply(lambda row: row['temperature'] < AGE_THRESHOLDS[row['age_category']]['temp_low'], axis=1)

    # Respiratory Rate flags
    df['flag_rr_low'] = df.apply(lambda row: row['resp_rate'] < AGE_THRESHOLDS[row['age_category']]['rr_low'], axis=1)
    df['flag_rr_high'] = df.apply(lambda row: row['resp_rate'] >= AGE_THRESHOLDS[row['age_category']]['rr_high'], axis=1)

    # Heart Rate flags
    df['flag_hr_low'] = df.apply(lambda row: row['heart_rate'] < AGE_THRESHOLDS[row['age_category']]['hr_low'], axis=1)
    df['flag_hr_high'] = df.apply(lambda row: row['heart_rate'] >= AGE_THRESHOLDS[row['age_category']]['hr_high'], axis=1)

    # Blood Pressure flags
    df['flag_sbp_low'] = df.apply(lambda row: row['sbp'] < AGE_THRESHOLDS[row['age_category']]['sbp_low'], axis=1)
    df['flag_sbp_high'] = df.apply(lambda row: row['sbp'] >= AGE_THRESHOLDS[row['age_category']]['sbp_high'], axis=1)
    df['flag_dbp_low'] = df.apply(lambda row: row['dbp'] < (AGE_THRESHOLDS[row['age_category']]['sbp_low'] * 0.6), axis=1) # Estimate DBP low
    df['flag_dbp_high'] = df.apply(lambda row: row['dbp'] >= (AGE_THRESHOLDS[row['age_category']]['sbp_high'] * 0.6), axis=1) # Estimate DBP high

    return df

In [5]:
def compute_recent_trends_delta(df):
    """
    Computes trends for each vital by differencing consecutive readings.
    Applies stricter interpretation using age-specific thresholds.
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)  # handled separately
    }

    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        trends[f"{vital}_trend_flag"] = flag

    # Shock Index trend
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_HIGH:
                flag = "Shock Index high — improving" if avg_si_delta < 0 else "Shock Index high — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            trends['shock_index_trend_flag'] = flag

    return trends


In [6]:
def detect_hypotensive_shock(df, shock_window='2h', sbp_sustainment=0.8, hr_sustainment=0.5):
    """
    A lean function to detect hypotensive shock. Designed to integrate with other pipelines.
    Leverages the existing `apply_vital_range_flags` and `compute_recent_trends_delta` functions.

    Args:
        df (pd.DataFrame): Input DataFrame with columns: timestamp, sbp, dbp, heart_rate, age.
        shock_window (str): Pandas offset string for the rolling window (e.g., '2H').
        sbp_sustainment (float): Min fraction of time SBP must be low (e.g., 0.8 for 80%).
        hr_sustainment (float): Min fraction of time HR must be high (e.g., 0.5 for 50%).

    Returns:
        dict: A result dictionary containing the shock status, trends, and a message.
            {
                "shock_detected": True/False,
                "shock_status": "CRITICAL", "WARNING", "MONITOR", "NORMAL",
                "trends": {...}, # Output from compute_recent_trends_delta
                "message": "Clinical message for alerting"
            }
    """
    # 0. Input Check
    if df.empty:
        return {"shock_detected": False, "shock_status": "NORMAL", "trends": {}, "message": "Insufficient data"}

    df = df.copy()
    
    # 1. APPLY EXISTING FUNCTION: Get basic flags (flag_sbp_low, flag_hr_high)
    df = apply_vital_range_flags(df)
    
    # 2. Calculate Shock Index (core derived metric for shock)
    df['shock_index'] = df['heart_rate'] / df['sbp'].replace(0, 0.001)
    df['flag_si_high'] = df['shock_index'] >= 0.9 # THRESH_SHOCK_INDEX_HIGH

    # 3. Core Detection Logic: Use time-aware rolling windows
    # Ensure we have a datetime index for rolling
    if 'timestamp' in df.columns:
        df = df.set_index('timestamp').sort_index()
    
    # Calculate sustained conditions over the shock window
    df['pct_sbp_low'] = df['flag_sbp_low'].rolling(shock_window, min_periods=1).mean()
    df['pct_hr_high'] = df['flag_hr_high'].rolling(shock_window, min_periods=1).mean()
    df['any_si_high'] = df['flag_si_high'].rolling(shock_window, min_periods=1).max()
    
    # The Rule: SBP low + HR high + SI high, sustained
    shock_detected = (df['pct_sbp_low'] >= sbp_sustainment) & \
                     (df['pct_hr_high'] >= hr_sustainment) & \
                     (df['any_si_high'] == 1)
    
    
    # Get the current status from the last row
    current_shock = shock_detected.iloc[-1] if not shock_detected.empty else False
    
    # 4. APPLY EXISTING FUNCTION: Get recent trends for context
    trend_data = compute_recent_trends_delta(df.reset_index().tail(8)) # Get last 8 readings for trend

    # 5. Determine Status Level and Create Message
    if current_shock:
        status = "CRITICAL"
        message = f"CRITICAL: Hypotensive Shock detected. SBP <90 & HR>100 sustained over {shock_window}."
    else:
        # Check for emerging warning signs using the trends
        sbp_trend = trend_data.get('sbp_trend', 0)
        si_trend = trend_data.get('shock_index_trend', 0)
        
        # If SBP is trending down and SI is trending up, it's a warning
        if sbp_trend < 0 and si_trend > 0:
            status = "WARNING"
            message = f"WARNING: Trends indicate deteriorating hemodynamics. SBP falling, Shock Index rising."
        elif df['flag_sbp_low'].iloc[-1] or df['flag_si_high'].iloc[-1]:
            status = "MONITOR"
            message = f"MONITOR: Isolated hypotension or elevated Shock Index present."
        else:
            status = "NORMAL"
            message = "NORMAL: No signs of shock."

    # Add trend info to the message
    trend_msg = trend_data.get('sbp_trend_flag', '')
    if trend_msg:
        message += f" Trend: {trend_msg}"

    return {
        "shock_detected": current_shock,
        "shock_status": status,
        "trends": trend_data,
        "message": message
    }

In [7]:
import pandas as pd
from datetime import datetime, timedelta

# Generate synthetic patient data
timestamps = [datetime.now() - timedelta(minutes=10*i) for i in range(6)][::-1]  # last 2 hours, 10 min intervals

data = {
    "timestamp": timestamps,
    "age": [45]*6,
    "sbp": [80, 75, 85, 80, 65, 85],  # SBP falling <90
    "dbp": [70, 65, 70, 60, 50, 60],     # DBP trending low
    "heart_rate": [111, 120, 110, 100, 120, 100],  # rising HR
    "resp_rate": [24, 25, 25, 24, 27, 24],  # slightly elevated RR
    "spo2": [98, 100, 100, 98, 98, 100],    # dropping SpO₂
    "temperature": [35]*6
}

df_test = pd.DataFrame(data)
# Run your detection
result = detect_hypotensive_shock(df_test)
print(result)

{'shock_detected': np.True_, 'shock_status': 'CRITICAL', 'trends': {'resp_rate_trend': np.float64(0.0), 'resp_rate_trend_flag': 'Abnormal and worsening', 'heart_rate_trend': np.float64(-2.2), 'heart_rate_trend_flag': 'Normal but deteriorating', 'sbp_trend': np.float64(1.0), 'sbp_trend_flag': 'Still abnormal — but improving', 'temperature_trend': np.float64(0.0), 'temperature_trend_flag': 'Abnormal and worsening', 'spo2_trend': np.float64(0.4), 'spo2_trend_flag': 'Normal and stable', 'shock_index_trend': np.float64(-0.042), 'shock_index_trend_flag': 'Shock Index high — improving'}, 'message': 'CRITICAL: Hypotensive Shock detected. SBP <90 & HR>100 sustained over 2h. Trend: Still abnormal — but improving'}
